In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

backup_write("src/careeragent/langgraph/runtime_nodes.py", r'''
from __future__ import annotations

from typing import Any, Callable, Dict, Optional


def _apply_delta(state: Dict[str, Any], delta: Any) -> Dict[str, Any]:
    """
    Description: Apply LangGraph-style deltas OR full-state returns safely.
    Layer: L6
    Input: state + delta/full_state
    Output: updated state
    """
    if not isinstance(delta, dict):
        return state

    # If a node returned a full state snapshot, accept it
    if "run_id" in delta and "status" in delta and ("live_feed" in delta or "meta" in delta):
        state.update(delta)
        return state

    # list reducers
    for k in ("live_feed", "attempts", "gates", "evaluations", "steps"):
        if k in delta and isinstance(delta[k], list):
            state.setdefault(k, [])
            state[k].extend(delta[k])

    # dict reducers
    if "artifacts" in delta and isinstance(delta["artifacts"], dict):
        state.setdefault("artifacts", {})
        state["artifacts"].update(delta["artifacts"])

    # overwrite other keys
    for k, v in delta.items():
        if k in ("live_feed", "attempts", "gates", "evaluations", "steps", "artifacts"):
            continue
        state[k] = v

    return state


def _feed(state: Dict[str, Any], layer: str, agent: str, message: str) -> None:
    state.setdefault("live_feed", [])
    state["live_feed"].append({"layer": layer, "agent": agent, "message": message})


def _fast_l0_guard(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: Guaranteed-fast local security gate. No network calls, no heavy imports.
    Layer: L0
    Input: resume_text
    Output: delta
    """
    resume = (state.get("resume_text") or "").lower()
    signals = [
        "ignore previous instructions",
        "system prompt",
        "developer message",
        "jailbreak",
        "exfiltrate",
        "reveal hidden",
    ]
    blocked = any(s in resume for s in signals)

    attempt = {
        "layer_id": "L0",
        "agent": "SanitizeAgent",
        "tool": "local.prompt_injection_heuristic",
        "model": None,
        "status": "failed" if blocked else "ok",
        "confidence": 0.10 if blocked else 0.95,
        "error": "prompt_injection" if blocked else None,
    }

    msg = "Security passed." if not blocked else "Blocked: prompt injection detected."

    delta: Dict[str, Any] = {
        "live_feed": [{"layer": "L0", "agent": "SanitizeAgent", "message": msg}],
        "attempts": [attempt],
    }
    if blocked:
        delta["status"] = "blocked"
        delta["pending_action"] = "security_blocked"
    return delta


def _pick_fn(mod: Any, *names: str) -> Optional[Callable[..., Any]]:
    """
    Description: Pick the first existing callable from a list of candidate names.
    Layer: L6
    """
    for n in names:
        fn = getattr(mod, n, None)
        if callable(fn):
            return fn
    return None


async def run_single_layer(state: Dict[str, Any], layer: str) -> Dict[str, Any]:
    """
    Description: Execute one layer for background runner + Engineer View.
    Layer: L6
    Input: state + layer_id
    Output: updated state
    """
    layer = (layer or "").upper().strip()

    # ✅ L0 is handled locally (instant, cannot hang)
    if layer == "L0":
        return _apply_delta(state, _fast_l0_guard(state))

    # L2–L5: call your existing node functions from nodes.py (name-flexible)
    if layer in ("L2", "L3", "L4", "L5"):
        from careeragent.langgraph import nodes as nodes_mod

        l2 = _pick_fn(nodes_mod, "l2_parser_node", "l2_parse_node", "l2_intake_node", "parser_node")
        l3 = _pick_fn(nodes_mod, "l3_discovery_node", "discovery_node", "hunt_node")
        l4 = _pick_fn(nodes_mod, "l4_match_node", "matcher_node", "match_node", "score_node")
        l5 = _pick_fn(nodes_mod, "l5_rank_node", "rank_node", "evaluator_rank_node")

        mapping = {"L2": l2, "L3": l3, "L4": l4, "L5": l5}
        fn = mapping.get(layer)

        if not fn:
            _feed(state, "L1", "Runtime", f"{layer} node not found in careeragent.langgraph.nodes.py")
            state["status"] = "needs_human_approval"
            state["pending_action"] = f"missing_{layer.lower()}_node"
            return state

        delta = await fn(state)  # type: ignore[misc]
        state = _apply_delta(state, delta)

        if layer == "L5" and state.get("ranking") and not state.get("pending_action"):
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"

        return state

    # L6–L9 if your nodes_l6_l9.py exists
    if layer in ("L6", "L7", "L8", "L9"):
        from careeragent.langgraph.nodes_l6_l9 import (
            l6_draft_node, l6_evaluator_node,
            l7_apply_node, l7_evaluator_node,
            l8_tracker_node, l8_evaluator_node,
            l9_analytics_node,
        )

        if layer == "L6":
            state = _apply_delta(state, await l6_draft_node(state))      # type: ignore[arg-type]
            state = _apply_delta(state, await l6_evaluator_node(state))  # type: ignore[arg-type]
            return state
        if layer == "L7":
            state = _apply_delta(state, await l7_apply_node(state))      # type: ignore[arg-type]
            state = _apply_delta(state, await l7_evaluator_node(state))  # type: ignore[arg-type]
            return state
        if layer == "L8":
            state = _apply_delta(state, await l8_tracker_node(state))      # type: ignore[arg-type]
            state = _apply_delta(state, await l8_evaluator_node(state))    # type: ignore[arg-type]
            return state
        if layer == "L9":
            state = _apply_delta(state, await l9_analytics_node(state))    # type: ignore[arg-type]
            return state

    _feed(state, "L1", "Runtime", f"Layer {layer} not implemented.")
    return state


async def approve_ranking_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: HITL approve ranking -> generate drafts (L6) then pause.
    Layer: L6
    """
    from careeragent.langgraph.nodes_l6_l9 import l6_draft_node, l6_evaluator_node

    state["status"] = "running"
    state["pending_action"] = None
    _feed(state, "L6", "HITL", "Ranking approved. Generating drafts…")

    state = _apply_delta(state, await l6_draft_node(state))      # type: ignore[arg-type]
    state = _apply_delta(state, await l6_evaluator_node(state))  # type: ignore[arg-type]

    state["status"] = "needs_human_approval"
    state["pending_action"] = "review_drafts"
    return state


async def approve_drafts_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: HITL approve drafts -> apply + track + analytics -> completed.
    Layer: L7-L9
    """
    from careeragent.langgraph.nodes_l6_l9 import (
        l7_apply_node, l7_evaluator_node,
        l8_tracker_node, l8_evaluator_node,
        l9_analytics_node,
    )

    state["status"] = "running"
    state["pending_action"] = None
    _feed(state, "L7", "HITL", "Drafts approved. Applying + tracking + analytics…")

    state = _apply_delta(state, await l7_apply_node(state))      # type: ignore[arg-type]
    state = _apply_delta(state, await l7_evaluator_node(state))  # type: ignore[arg-type]
    if state.get("status") == "needs_human_approval":
        return state

    state = _apply_delta(state, await l8_tracker_node(state))      # type: ignore[arg-type]
    state = _apply_delta(state, await l8_evaluator_node(state))    # type: ignore[arg-type]
    if state.get("status") == "needs_human_approval":
        return state

    state = _apply_delta(state, await l9_analytics_node(state))    # type: ignore[arg-type]

    state["status"] = "completed"
    state["pending_action"] = None
    _feed(state, "L9", "HITL", "Run completed.")
    return state
''')

print("✅ runtime_nodes.py patched: L0 is now instant + L2–L5 imported by flexible names.")
print("Restart backend and start a NEW run.")